In [ ]:
# Third-party
import astropy.coordinates as coord
from astropy.table import Table
import astropy.units as u
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

import gala.coordinates as gc
import gala.dynamics as gd
import gala.potential as gp
from gala.units import galactic
from pyia import GaiaData

Omega cen cluster phase-space coordinates from Eugene Vasiliev:

In [ ]:
globclust = Table.read('/Users/adrian/data/Misc/Vasiliev-globclust.txt', format='ascii.fixed_width')
cs = coord.SkyCoord(ra=globclust['RA']*u.deg, 
                    dec=globclust['DEC']*u.deg,
                    distance=globclust['D']*u.kpc,
                    pm_ra_cosdec=globclust['PMRA']*u.mas/u.yr,
                    pm_dec=globclust['PMDEC']*u.mas/u.yr,
                    radial_velocity=globclust['Vlos']*u.km/u.s)

In [ ]:
c = cs[globclust['Name'] == 'NGC 5139 (omega Cen)']
c

In [ ]:
galcen_frame = coord.Galactocentric(galcen_distance=8.1*u.kpc)

Generate Galactocentric orbital initial conditions for Omega cen:

In [ ]:
w0 = gd.PhaseSpacePosition(c.transform_to(galcen_frame).data)

Define a mass model for the Milky Way:

In [ ]:
mw = gp.MilkyWayPotential()
H = gp.Hamiltonian(mw)
for k in mw:
    print(repr(mw[k]))

Integrate the orbit of Omega Cen backwards in time:

In [ ]:
orbit = H.integrate_orbit(w0, dt=-0.5, t1=0, t2=-1*u.Gyr)
_ = orbit.plot()

Now generate a mock stream going forward in time:

In [ ]:
from gala.dynamics.mockstream import fardal_stream

In [ ]:
stream = fardal_stream(H, orbit[::-1], prog_mass=5e5*u.Msun, release_every=1)

In [ ]:
_ = stream.plot()

In [ ]:
stream_icrs = stream.to_coord_frame(coord.ICRS, galactocentric_frame=galcen_frame)

In [ ]:
style = dict(marker='o', ls='none', alpha=0.5, mew=0, ms=2.5)

fig, axes = plt.subplots(4, 1, figsize=(10, 10), 
                         sharex=True)

ax = axes[0]
ax.plot(stream_icrs.ra.degree,
        stream_icrs.dec.value,
        **style)
ax.set_ylim(-70, -10)
ax.set_ylabel(r'$\delta$')

ax = axes[1]
ax.plot(stream_icrs.ra.degree,
        stream_icrs.pm_ra_cosdec.value,
        **style)
ax.set_ylabel(r'$\mu_\alpha$')

ax = axes[2]
ax.plot(stream_icrs.ra.degree,
        stream_icrs.pm_dec.value,
        **style)
ax.set_ylim(-15, 0)
ax.set_ylabel(r'$\mu_\delta$')

ax = axes[3]
ax.plot(stream_icrs.ra.degree,
        stream_icrs.radial_velocity.value,
        **style)
ax.set_ylim(-10, 300)
ax.set_ylabel('RV [km/s]')

ax.set_xlim(240, 180)
ax.set_xlabel(r'$\alpha$')